Based on the examples provided, make your own class for implementing locally weighted regression to work with multiple features, and also train and test data. Show an application to a real data set with your implementation, and present the 10-fold cross-validated mean square error.

In [ ]:
# computational libraries
import numpy as np
import pandas as pd
import xgboost
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler, QuantileTransformer, MinMaxScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from scipy.spatial import Delaunay
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import scipy.stats as stats
from sklearn.model_selection import train_test_split as tts, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error as mse
from scipy.spatial.distance import cdist
from scipy.interpolate import interp1d, RegularGridInterpolator, griddata, LinearNDInterpolator, NearestNDInterpolator
from math import ceil
from scipy import linalg
# the following line(s) are necessary if you want to make SKlearn compliant functions
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
import warnings
warnings.filterwarnings('ignore')

from sklearn import linear_model
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression

In [ ]:
std_scale = StandardScaler()
quant_scale = QuantileTransformer()
minmax_scale = MinMaxScaler()

scalers = [std_scale, quant_scale, minmax_scale]

In [ ]:
# Gaussian Kernel
def Gaussian(w):
  return np.where(w>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*w**2))

# Tricubic Kernel
def Tricubic(w):
  return np.where(w>1,0,70/81*(1-w**3)**3)

# Quartic Kernel
def Quartic(w):
  return np.where(w>1,0,15/16*(1-w**2)**2)

# Epanechnikov Kernel
def Epanechnikov(w):
  return np.where(w>1,0,3/4*(1-w**2))

In [ ]:
def weight_function(u,v,kern=Gaussian,tau=0.5):
    return kern(cdist(u, v, metric='euclidean')/(2*tau))

In [ ]:
# # here we have a function that computes the Euclidean distance between all the observations in u, and v
# def dist(u,v):
#   return cdist(u, v, metric='euclidean')

# # how about the case when v is a matrix?
# def dist(u,v):
#   D = []
#   if len(v.shape)==1:
#     v = v.reshape(1,-1)
#   # we would like all the pairwise combinations if u and v are matrices
#   # we could avoid two for loops if we consider broadcasting
#   for rowj in v:
#     D.append(np.sqrt(np.sum((u-rowj)**2,axis=1)))
#   return np.array(D).T

# here we have a function that computes the Euclidean distance between all the observations in u, and v
def dist(u,v):
  if len(v.shape)==1:
    v = v.reshape(1,-1)
  d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
  return d

In [ ]:
def lw_ag_md(x, y, xnew,f=2/3,iter=3, intercept=True):

  n = len(x)
  r = int(ceil(f * n))
  yest = np.zeros(n)

  if len(y.shape)==1: # here we make column vectors
    y = y.reshape(-1,1)

  if len(x.shape)==1:
    x = x.reshape(-1,1)

  if intercept:
    x1 = np.column_stack([np.ones((len(x),1)),x])
  else:
    x1 = x

  h = [np.sort(np.sqrt(np.sum((x-x[i])**2,axis=1)))[r] for i in range(n)]
  # dist(x,x) is always symmetric
  w = np.clip(dist(x,x) / np.array(h), 0.0, 1.0)
  # note that w is a square matrix and in Python arithmetic operations such as
  # w**3 or 1-w**3 are performed element-wise
  #w = (1-w**3)**3 # a Tricubic kernel
  w = Epanechnikov(w)

  #Looping through all X-points
  delta = np.ones(n)
  for iteration in range(iter):
    for i in range(n):
      W = np.diag(delta).dot(np.diag(w[i,:]))
      # when we multiply two diagonal matrices we get also a diagonal matrix
      b = np.transpose(x1).dot(W).dot(y)
      A = np.transpose(x1).dot(W).dot(x1)
      ##
      A = A + 0.0001*np.eye(x1.shape[1]) # if we want L2 regularization for solving the system
      beta = linalg.solve(A, b)

      #beta, res, rnk, s = linalg.lstsq(A, b)
      yest[i] = np.dot(x1[i],beta.ravel())

    residuals = y.ravel() - yest
    s = np.median(np.abs(residuals))

    delta = np.clip(residuals / (6.0 * s), -1, 1)

    delta = (1 - delta ** 2) ** 2

  # here we are making predictions for xnew by using an interpolation and the predictions we made for the train data
  if x.shape[1]==1:
    f = interp1d(x.flatten(),yest,fill_value='extrapolate')
    output = f(xnew)
  else:
    output = np.zeros(len(xnew))
    for i in range(len(xnew)):
      ind = np.argsort(np.sqrt(np.sum((x-xnew[i])**2,axis=1)))[:r]
      pca = PCA(n_components=3)
      x_pca = pca.fit_transform(x[ind])
      tri = Delaunay(x_pca,qhull_options='QJ Pp')
      f = LinearNDInterpolator(tri,yest[ind])
      input_i = pca.transform(xnew[i].reshape(1,-1))
      output[i] = f(input_i)
      # the output may have NaN's where the data points from xnew are outside the convex hull of X

  if sum(np.isnan(output))>0:
    g = NearestNDInterpolator(x,yest.ravel())
    # output[np.isnan(output)] = g(X[np.isnan(output)])
    output[np.isnan(output)] = g(xnew[np.isnan(output)])
  return output

In [ ]:
#Lowess Class without triangulation

class Lowess:
    def __init__(self, kernel = Gaussian, tau=0.05):
        self.kernel = kernel
        self.tau = tau

    def fit(self, x, y):
        kernel = self.kernel
        tau = self.tau
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        lm = linear_model.Ridge(alpha=0.0001)
        w = weight_function(x,x_new,self.kernel,self.tau)

        if np.isscalar(x_new):
          lm.fit(np.diag(w)@(x.reshape(-1,1)),np.diag(w)@(y.reshape(-1,1)))
          yest = lm.predict([[x_new]])[0][0]
        else:
          n = len(x_new)
          yest_test = []
          #Looping through all x-points
          for i in range(n):
            lm.fit(np.diag(w[:,i])@x,np.diag(w[:,i])@y)
            yest_test.append(lm.predict([x_new[i]]))
        return np.array(yest_test).flatten()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('drive/MyDrive/concrete.csv')

In [ ]:
data

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [ ]:
x = data.loc[:,'cement':'age'].values
y = data['strength'].values

In [ ]:
mse_lwr = []
mse_rf = []
scale = std_scale
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
#model_rf = RandomForestRegressor(n_estimators=200,max_depth=7)
model_rf = XGBRegressor(objective ='reg:squarederror',n_estimators=100,reg_lambda=20,alpha=1,gamma=10,max_depth=7)
# model_lw = Lowess_AG_MD(f=1/3,iter=2,intercept=True)
model_1 = Lowess(kernel=Gaussian,tau=0.4)
model_2 = Lowess(kernel=Tricubic,tau=0.3)
#model_1 = Lowess_AG_MD(f=1/4,iter=2,intercept=True)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain].ravel()
  ytest = y[idxtest].ravel()
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_1.fit(xtrain,ytrain)
  yhat_train = model_1.predict(xtrain)
  residuals_train = ytrain - yhat_train
  model_2.fit(xtrain,residuals_train)
  residuals_hat = model_2.predict(xtest)
  yhat_lw = model_1.predict(xtest) + model_2.predict(xtest)

  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for a DT-based method: '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 42.14569000966465
The Cross-validated Mean Squared Error for a DT-based method: 20.18308623182933


In [ ]:
mse_lwr = []
mse_rf = []
scale = quant_scale
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
#model_rf = RandomForestRegressor(n_estimators=200,max_depth=7)
model_rf = XGBRegressor(objective ='reg:squarederror',n_estimators=100,reg_lambda=20,alpha=1,gamma=10,max_depth=7)
# model_lw = Lowess_AG_MD(f=1/3,iter=2,intercept=True)
model_1 = Lowess(kernel=Gaussian,tau=0.4)
model_2 = Lowess(kernel=Tricubic,tau=0.3)
#model_1 = Lowess_AG_MD(f=1/4,iter=2,intercept=True)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain].ravel()
  ytest = y[idxtest].ravel()
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_1.fit(xtrain,ytrain)
  yhat_train = model_1.predict(xtrain)
  residuals_train = ytrain - yhat_train
  model_2.fit(xtrain,residuals_train)
  residuals_hat = model_2.predict(xtest)
  yhat_lw = model_1.predict(xtest) + model_2.predict(xtest)

  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for a DT-based method: '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 20.84908646431679
The Cross-validated Mean Squared Error for a DT-based method: 20.18308623182933


In [ ]:
mse_lwr = []
mse_rf = []
scale = minmax_scale
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
#model_rf = RandomForestRegressor(n_estimators=200,max_depth=7)
model_rf = XGBRegressor(objective ='reg:squarederror',n_estimators=100,reg_lambda=20,alpha=1,gamma=10,max_depth=7)
# model_lw = Lowess_AG_MD(f=1/3,iter=2,intercept=True)
model_1 = Lowess(kernel=Gaussian,tau=0.4)
model_2 = Lowess(kernel=Tricubic,tau=0.3)
#model_1 = Lowess_AG_MD(f=1/4,iter=2,intercept=True)

for idxtrain, idxtest in kf.split(x):
  xtrain = x[idxtrain]
  ytrain = y[idxtrain].ravel()
  ytest = y[idxtest].ravel()
  xtest = x[idxtest]
  xtrain = scale.fit_transform(xtrain)
  xtest = scale.transform(xtest)

  model_1.fit(xtrain,ytrain)
  yhat_train = model_1.predict(xtrain)
  residuals_train = ytrain - yhat_train
  model_2.fit(xtrain,residuals_train)
  residuals_hat = model_2.predict(xtest)
  yhat_lw = model_1.predict(xtest) + model_2.predict(xtest)

  model_rf.fit(xtrain,ytrain)
  yhat_rf = model_rf.predict(xtest)

  mse_lwr.append(mse(ytest,yhat_lw))
  mse_rf.append(mse(ytest,yhat_rf))
print('The Cross-validated Mean Squared Error for Locally Weighted Regression is : '+str(np.mean(mse_lwr)))
print('The Cross-validated Mean Squared Error for a DT-based method: '+str(np.mean(mse_rf)))

The Cross-validated Mean Squared Error for Locally Weighted Regression is : 45.368275750156776
The Cross-validated Mean Squared Error for a DT-based method: 20.18308623182933


In [ ]:
XGB_MSE=[]
for scale in scalers:
  for idxtrain, idxtest in kf.split(x):
    xtrain = x[idxtrain]
    ytrain = y[idxtrain].ravel()
    ytest = y[idxtest].ravel()
    xtest = x[idxtest]
    xtrain = scale.fit_transform(xtrain)
    xtest = scale.transform(xtest)

  model_xgboost = xgboost.XGBRFRegressor()
  model_xgboost.fit(xtrain,ytrain)
  XGB_MSE.append(mse(ytest,model_xgboost.predict(xtest)))
print(XGB_MSE)

[32.5773262908273, 32.5773262908273, 32.5773262908273]


Since we are only looking at the MSE of the LWR, we see that the middle trial, namely, the one using the scale = QuantileTransformer() along with the Tricubic and Gaussian models achieves a MSE of 20.85 which is lower than the eXtream Gradient Boosting MSE of 32.58.